In [1]:
import numpy as np

In [34]:
class Node:
    def __init__(self, value, prob):
        self.value = value
        self.prob = prob
        self.children = []

    def add_child(self, child):
        self.children.append((child))

In [261]:
root = Node('A', 1.0)
B = Node('B', 0.5)
C = Node('C', 0.2)
D = Node('D', 0.3)

root.add_child(B)
root.add_child(C)
root.add_child(D)

B.add_child(Node('E', 0.15))
B.add_child(Node('F', 0.08))
B.add_child(Node('G', 0.06))
C.add_child(Node('H', 0.07))
C.add_child(Node('I', 0.05))
C.add_child(Node('J', 0.40))
D.add_child(Node('K', 0.02))
D.add_child(Node('L', 0.16))
D.add_child(Node('M', 0.10))



In [97]:
def print_tree(node, depth=0):
    indent = "  " * depth
    print(f"{indent}- {node.value} (Prob: {node.prob})")
    for child in node.children:
        print_tree(child, depth + 1)

In [ ]:
def stochastic_beam_search(root, k, steps, temperature=1):
    beam = [(root, 0.0, [root.value])]  # (node, score, sequence)

    for _ in range(steps):
        expansions = []
        for node,score,sequence in beam:
            if node == None:
                continue

            children = node.children
            # phi_s'
            g = [np.log(node.prob) + np.log(i.prob) for i in children]
            # gumbel = -np.log(-np.log(np.random.uniform(size = len(children))))
            # G_phi_s'
            gumbel_score = np.array(g)/temperature + np.random.gumbel(size=len(children))
            Z = max(gumbel_score)
            # ~G_phi_s'
            gumbel_score = -np.log(np.exp(-score)-np.exp(-Z)+np.exp(-gumbel_score))
            expansions += [(c, s, sequence+[c.value]) for c,s in zip(children, gumbel_score)]
            
        ordered_expansions = sorted(expansions, key=lambda x: x[1], reverse=True)
        beam = ordered_expansions[:k]

    return beam

In [262]:
print_tree(root)

- A (Prob: 1.0)
  - B (Prob: 0.5)
    - E (Prob: 0.15)
    - F (Prob: 0.08)
    - G (Prob: 0.06)
  - C (Prob: 0.2)
    - H (Prob: 0.07)
    - I (Prob: 0.05)
    - J (Prob: 0.4)
  - D (Prob: 0.3)
    - K (Prob: 0.02)
    - L (Prob: 0.16)
    - M (Prob: 0.1)


In [264]:
beam = stochastic_beam_search(root, k=4, steps=2, temperature=0.5)
for  _, score, sequence in beam:
    print(f"Sequence: {' '.join(sequence)}, Score: {score}")

Sequence: A B E, Score: -0.0
Sequence: A D L, Score: -2.00273646800278
Sequence: A C J, Score: -2.0225193653121987
Sequence: A B F, Score: -5.9557109983521475


In [302]:
root1 = Node('|', 1.0)

B = Node('today', 0.6)
C = Node('yesterday', 0.2)
D = Node('tomorrow', 0.1)
E = Node('tmr', 0.1)

root1.add_child(B)
root1.add_child(C)
root1.add_child(D)

F = Node('am', 0.05)
G = Node('is', 0.35)
H = Node('are', 0.15)
I = Node('was', 0.15)
J = Node('were', 0.05)
K = Node('do', 0.1)
L = Node('will be', 0.15)

M = Node('monday', 0.1)
N = Node('tuesday', 0.2)
O = Node('wednesday', 0.1)
P = Node('thursday', 0.2)
Q = Node('friday', 0.1)
R = Node('saturday', 0.2)
S = Node('sunday', 0.1)

for i in [M, N, O, P, Q, R, S]:
    i.add_child(Node('. |', 0.4))
    i.add_child(Node('? |', 0.3))
    i.add_child(Node('! |', 0.3))

for i in [M, N, O, P, Q, R, S]:
    for j in [F, G, H, I, J, K, L]:
        j.add_child(i)

for i in [F, G, H, I, J, K, L]:
    for j in [B, C, D]:
        j.add_child(i)


In [307]:
beam = stochastic_beam_search(root1, k=4, steps=4, temperature=0.6)
for  _, score, sequence in beam:
    print(f"Sequence: {' '.join(sequence)}, Score: {score}")

Sequence: | tomorrow is saturday ! |, Score: -0.0
Sequence: | today is wednesday . |, Score: -0.9968667098974788
Sequence: | yesterday do thursday . |, Score: -1.6173052867977677
Sequence: | tomorrow is saturday . |, Score: -1.8563835038826522
